In [3]:
import torch
from torch import nn
from torch_geometric.nn import GCNConv, GATConv, ChebConv, SAGEConv
from torch.nn import Linear
import torch.nn.functional as F
from GNNNestedCVEvaluationBinary import GNNNestedCVEvaluation
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from hyperopt import hp
import numpy as np
from tqdm.notebook import tqdm
from torch_geometric.data import Data

In [ ]:
!pip install hyperopt
!pip install tqdm
!pip install numpy
!pip install torch
!pip install torch_geometric

In [ ]:
## TODO
def get_complete_node_table_from_sql(dataset_name) -> np.array [#number of rows x # number of features(50)]:
    connector = sql_connector(dataset_name)
    data = connector.execute_query("SELECT * FROM node_table")
    data = remove_index_and_primary_keys(data)
    return np.array(data) 

def get_complete_edge_index_table_from_sql(dataset_name) -> np.array [2 # number of edges]:
    return edge_index

node_features = get_complete_node_table_from_sql("1000_nodes_dataset")
node_labels = node_features[:, -1]
edge_index = get_complete_edge_index_table_from_sql("1000_nodes_dataset")

In [15]:
## Working example
# num_nodes = 50
# num_features = 10
# num_edges = 10
# node_features = torch.rand(num_nodes, num_features)
# node_labels = torch.randint(0, 2, (num_nodes,))
# edge_index = torch.randint(0, num_nodes, (2, num_edges))

# node_features = node_features.numpy()
# node_labels = node_labels.numpy()
# edge_index = edge_index.numpy()

In [16]:
data = Data(x = torch.from_numpy(node_features).type(torch.float), y = torch.from_numpy(node_labels).type(torch.float), edge_index = torch.from_numpy(edge_index).type(torch.long))
data

Data(x=[50, 10], edge_index=[2, 10], y=[50])

In [17]:
## Example data
dataset_name = 'Cora'
split = "public"
dataset = Planetoid(root='data/', name=dataset_name, split=split)
dataset.transform = T.NormalizeFeatures()
data_example = dataset[0]
data_example

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [18]:
class GCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout = .2, normalize = False, add_self_loops = True):
        super(GCN, self).__init__()
        hidden_dim = int(hidden_dim)        
        self.conv1 = GCNConv(in_dim, hidden_dim, normalize = normalize, add_self_loops=add_self_loops)
        self.conv2 = GCNConv(hidden_dim, out_dim, normalize = normalize, add_self_loops=add_self_loops)
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

In [19]:
class GNNSpace():
    def __init__(self, num_classes):
        self.hidden_dim_limits = [128]
        self.dropout_limits = [0.0]
        self.weight_decay_limits = [1e-5]
        self.lr_limits = [3e-4]
        self.out_dim = [num_classes]
        self.gnn_space = None
        self.initialize_space()
        

    def initialize_space(self):
        gnn_choices = {
            'out_dim': self.out_dim,
            'hidden_dim': self.hidden_dim_limits,
            'lr': self.lr_limits,
            'weight_decay': self.weight_decay_limits,
            'dropout': self.dropout_limits
        }
         
        self.gnn_space = {
            **{key: hp.choice(key, value) for key, value in gnn_choices.items()}
        }
        
    def add_choice(self, key, items):
        self.gnn_space[key] = hp.choice(key, items)
        
    def add_uniform(self, key, limits: tuple):
        self.gnn_space[key] = hp.uniform(key, limits[0], limits[1])
        
    def add_loguniform(self, key, limits: tuple):
        self.gnn_space[key] = hp.loguniform(key, np.log(limits[0]), np.log(limits[1]))
        
    def add_qloguniform(self, key, limits, q):
        self.gnn_space[key] = hp.qloguniform(key, low=np.log(limits[0]), high=np.log(limits[1]), q=q)

class GCNSpace(GNNSpace):
    def __init__(self, dataset):
        super().__init__(dataset)

    def get_space(self):
        self.add_choice('normalize', [True])
        self.add_choice('add_self_loops', [True])
        return self.gnn_space      

In [20]:
device = torch.device("cpu")

In [21]:
gcn_space = GCNSpace(1)
space = gcn_space.get_space()

In [22]:
gnn_nestedCV_evaluation = GNNNestedCVEvaluation(device, GCN,data, max_evals= 1)
gnn_nestedCV_evaluation.nested_cross_validate(5, 5, space)

0it [00:00, ?it/s]

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/11 09:52:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/11 09:53:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/11 09:53:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/07/11 09:53:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/07/11 09:53:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Total Trials: 1: 1 succeeded, 0 failed, 0 cancelled.                         


        Using a 5 x 5 nested StratifiedKFold Cross-Validation, we obtain:
        0.3600 +- 0.0490.

        self.outer_scores: [0.40000001 0.40000001 0.30000001 0.30000001 0.40000001]

        self.best_params_per_fold: [{'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}]

        self.best_models: [GCN(
  (conv1): GCNConv(10, 128)
  (conv2): GCNConv(128, 1)
  (dropout): Dropout(p=0.0, inplac

In [26]:
gnn_nestedCV_evaluation.nested_transd_cv


        Using a 5 x 5 nested StratifiedKFold Cross-Validation, we obtain:
        0.3600 +- 0.0490.

        self.outer_scores: [0.40000001 0.40000001 0.30000001 0.30000001 0.40000001]

        self.best_params_per_fold: [{'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}, {'add_self_loops': True, 'dropout': 0.0, 'hidden_dim': 128, 'lr': 0.0003, 'normalize': True, 'out_dim': 1, 'weight_decay': 1e-05}]

        self.best_models: [GCN(
  (conv1): GCNConv(10, 128)
  (conv2): GCNConv(128, 1)
  (dropout): Dropout(p=0.0, inplac